In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np

In [2]:
AAPL_stock = pd.read_csv("./dataset/stocks/AAPL.csv")
AAPL_stock = AAPL_stock.loc[:, ['time', 'close']]
AAPL_stock['time'] = pd.to_datetime(AAPL_stock['time'])

In [3]:
AAPL_stock.head()

,time,close
0,2022-10-31 04:00:00,153.34
1,2022-11-01 04:00:00,150.65
2,2022-11-02 04:00:00,145.03
3,2022-11-03 04:00:00,138.88
4,2022-11-04 04:00:00,138.38


In [4]:
AAPL_stock.dtypes

time     datetime64[ns]
close           float64
dtype: object

In [5]:
AAPL_sentiment = pd.read_csv("./dataset/polygon_title_sentiment_3/AAPL_title_sentiments.csv")


In [6]:
AAPL_sentiment.head()

,ticker,sentiment,sentiment_score,news_id,time,title,article_url
0,AAPL,neutral,0.000000,0d2a16f0328775bebe96ecbd57fb1ee7b98fae09d2ee73...,2024-10-30 08:06:00,Only 10 Companies Have Reached a Trillion-Doll...,https://www.fool.com/investing/2024/10/30/10-c...
1,AAPL,negative,-0.548489,55d38db6df60d647da21ac34f9942bfc7729d3196b10ff...,2024-10-29 13:09:00,Microsoft Earnings Preview: Will Azure Growth ...,https://www.investing.com/analysis/microsoft-e...
2,AAPL,neutral,0.000000,4ef47f7a1d56daecdf73f02eb2ceb206df1e629e6f04ba...,2024-10-29 09:45:00,1 Supercharged Vanguard Fund That Can Turn $27...,https://www.fool.com/investing/2024/10/29/1-su...
3,AAPL,neutral,0.000000,1358145bef572d9b1580795b261a011c39169815dae386...,2024-10-29 09:24:00,Warren Buffett Sold $80 Billion of Apple Stock...,https://www.fool.com/investing/2024/10/29/warr...
4,AAPL,positive,0.578031,ebed3bc762a748dd4eba4d38230efe87101b1c28ad4096...,2024-10-28 12:51:00,Nasdaq 100 Technical: Rising US Treasury Yield...,https://www.investing.com/analysis/nasdaq-100-...


In [7]:
AAPL_sentiment = AAPL_sentiment[AAPL_sentiment['ticker'] == 'AAPL']
AAPL_sentiment = AAPL_sentiment.loc[:, ['time', 'sentiment_score']]

In [8]:
AAPL_sentiment.tail()

,time,sentiment_score
9876,2022-09-23 10:20:08,0.000000
9877,2022-09-23 09:49:47,-0.966222
9878,2022-09-23 09:30:00,0.579299
9879,2022-09-23 08:50:32,-0.966438
9880,2022-09-23 04:23:10,0.000000


In [9]:
AAPL_sentiment.dtypes

time                object
sentiment_score    float64
dtype: object

In [10]:
df = pd.DataFrame(AAPL_sentiment)
df['time'] = pd.to_datetime(df['time'])

# Group by date and calculate weighted average for ticker_sentiment_score
df['date'] = df['time'].dt.date

df = df.groupby('date').apply(
    lambda x: pd.Series({
        'time': pd.Timestamp(f"{x['date'].iloc[0]} 04:00:00"),
        'sentiment_score': x['sentiment_score'].mean()
    })
).reset_index(drop=True)


In [11]:
df.head()

,time,sentiment_score
0,2022-09-23 04:00:00,-0.230271
1,2022-09-24 04:00:00,0.000000
2,2022-09-25 04:00:00,0.101683
3,2022-09-26 04:00:00,-0.121616
4,2022-09-27 04:00:00,0.035076


In [12]:
finalData = pd.merge(AAPL_stock, df, on='time')
finalData.columns = ["DateTime", 'Price', 'Sentiment_score']
finalData = finalData[['DateTime', 'Sentiment_score', 'Price']]

In [13]:
finalData.tail()

,DateTime,Sentiment_score,Price
325,2024-10-23 04:00:00,0.085070,230.76
326,2024-10-24 04:00:00,0.525094,230.57
327,2024-10-25 04:00:00,0.233975,231.41
328,2024-10-28 04:00:00,0.274372,233.40
329,2024-10-29 04:00:00,-0.182830,233.67


In [14]:
finalData.to_csv("AAPL_processed.csv", index=False)